# CSC4020 HW2 Programming
### @117020119 Jiang Jingxin 
## Spam Classification (Gaussian Naive Bayes)

In [216]:
from scipy import io 
import pandas as pd 
import numpy as np

In [97]:
def read_spam_data():
    raw_data = io.loadmat('spamData.mat')
    Xtrain = pd.DataFrame(raw_data['Xtrain'])
    ytrain = pd.Series(np.hstack(raw_data['ytrain']))
    Xtest = pd.DataFrame(raw_data['Xtest'])
    ytest = pd.Series(np.hstack(raw_data['ytest']))
    return Xtrain, ytrain, Xtest, ytest

In [98]:
Xtrain, ytrain, Xtest, ytest = read_spam_data()

---
## Exercise 8.1 Logistic Regression

**Loss Function**：$ f({\bf w})=NLL({\bf w})+\lambda {\bf w}^T{\bf w}= - \sum_{i=1}^N[y_i\log\mu_i+(1-y_i)\log(1-\mu_i)]+ \lambda\sum_{i=0}^k w_i^2$

**Gradient**: $g({\bf w}) = X^T({\bf \mu}-{\bf y})+2\lambda {\bf w}$

In [923]:
class LogisticRegression:
    def __init__(self, λ, lr = 1,alpha=0.5, beta=0.5, max_iter=10000, tol = 1e-3, 
                 add_intercept = True, transform = None):
        self.lr = lr # learning rate
        self.alpha = alpha # step size
        self.beta = beta # step size shrinking factor 
        self.max_iter = max_iter # maximal interations
        self.λ = λ # regularization term
        self.iters = 0 # number of iterations
        self.tol = tol # stopping criteria of backtracking line search
        self.add_intercept = add_intercept # add intercept to X by default 
        self.transform = transform # type of transformation: stnd, log or binary
        
    def _transform(self, X):
        if self.transform == "stnd": # standardize
            X = (X-X.mean())/X.std()
        elif self.transform == "log": # logarithmize
            X = np.log(X+0.1)
        elif self.transform == "binary": #binarize
            X = (X>0).astype(np.int32)
        if self.add_intercept:
            X = np.hstack([np.ones((X.shape[0],1)), X])
        return X
    
    def _sigmoid(self, a):
        return 1 / (1 + np.exp(-a))
    
    def _loss(self, y, μ, w):
        return (sum(-np.log(μ[y==1]))+sum(-np.log(1-μ[y==0])) + self.λ*sum(w**2)) / y.size
    
    def _gradient(self, X, y, μ, w):
        return (X.T@(μ-y)) / y.size + 2*self.λ*w / y.size
        
    def fit(self, X, y):
        self.converged = False
        
        X = self._transform(X)
        n, k = X.shape
        
        '''
        gradient descent with backtracking line search
        '''
        prev_w = self.w = np.zeros(k)
        μ = self._sigmoid(X@self.w)
        prev_loss = self.loss = self._loss(y, μ, self.w)
        gradient = self._gradient(X, y, μ, self.w)
        
        for i in range(self.max_iter):
            norm_gradient = np.sqrt(sum(gradient**2))
            if (norm_gradient < self.tol):
                self.converged = True
                break  
            
            self.iters += 1
            t = self.lr
            self.w = prev_w - t * gradient
            μ = self._sigmoid(X@self.w)                       
            self.loss = self._loss(y, μ, self.w)
            
            while (self.loss > prev_loss - self.alpha*t*norm_gradient**2):
                t = t*self.beta;
                self.w = prev_w - t*gradient
                μ = self._sigmoid(X@self.w)
                self.loss = self._loss(y, μ, self.w) 
                
            prev_w = self.w
            prev_loss = self.loss
            gradient = self._gradient(X, y, μ, self.w)
        
    def predict_prob(self, X):
        X = self._transform(X)
        return self._sigmoid(X@self.w)
        
    def predict(self, X):
        return (self.predict_prob(X)>0.5).astype(np.int32)
    
    def predict_error(self,X,y):
        return sum(self.predict(X) != y)/y.size

In [929]:
from tqdm import tqdm

'''
lambda parameter tuning using cross validation
'''
def CV(X, y, lambdas = 10**np.linspace(-2,1,10), transform = None, k = 10, seed = None):
    np.random.seed(seed)
    errors = np.zeros_like(lambdas)
    
    idx = np.arange(y.size) 
    np.random.shuffle(idx) 
    fold = np.array_split(idx,k) # split shuffled index into k folds

    # cross validation using k folds
    for i in tqdm(range(k)):
        for l, lam in enumerate(lambdas):
            test_idx = fold[i]
            train_idx = np.setdiff1d(idx, test_idx)
            mod = LogisticRegression(lam,transform = transform)
            mod.fit(X.loc[train_idx],y[train_idx])
            errors[l] += mod.predict_error(X.loc[test_idx],y[test_idx])
    errors /= k
    Lam = lambdas[np.argmin(errors)]
    return Lam

### a. Standardize

In [561]:
lam_stnd = CV(Xtrain,ytrain,transform="stnd")

100%|██████████| 10/10 [03:17<00:00, 19.76s/it]


In [927]:
lam_stnd

1.9306977288832496

In [908]:
clfLR_stnd = LogisticRegression(λ=lam_stnd, transform = "stnd")
clfLR_stnd.fit(Xtrain,ytrain)

1. **training error**:

In [895]:
clfLR_stnd.predict_error(Xtrain,ytrain)

0.07765089722675367

2. **test error**:

In [896]:
clfLR_stnd.predict_error(Xtest,ytest)

0.087890625

### b. Logarithmize

In [657]:
lam_log = CV(Xtrain,ytrain,transform="log")

100%|██████████| 10/10 [05:51<00:00, 35.12s/it]


In [922]:
lam_log

2.1544346900318834

In [915]:
clfLR_log = LogisticRegression(λ=lam_log,transform = "log")
clfLR_log.fit(Xtrain,ytrain)

1. **training error**:

In [898]:
clfLR_log.predict_error(Xtrain,ytrain)

0.052528548123980424

2. **test error**:

In [899]:
clfLR_log.predict_error(Xtest,ytest)

0.057291666666666664

### c. Binarize

In [662]:
lam_bin = CV(Xtrain,ytrain,transform="binary")

100%|██████████| 10/10 [00:42<00:00,  4.25s/it]


In [928]:
lam_bin

0.46415888336127786

In [904]:
mod = LogisticRegression(λ=lam_bin,transform = "binary")
mod.fit(Xtrain,ytrain)

1. **training error**:

In [905]:
mod.predict_error(Xtrain,ytrain)

0.06394779771615008

2. **test error**:

In [906]:
mod.predict_error(Xtest,ytest)

0.07356770833333333

---
## Exercise 8.2 Naive Bayes

### a. NaiveBayes

$\begin{aligned}h({\bf x}^n)&=\arg\max_c\hat P({\cal C}_c)\hat P({\bf x}^n|{\cal C}_c)=\arg\max_c \left(\log \hat P({\cal C}_c)+ \log \hat P({\bf x}^n|{\cal C}_c)\right)\\&=\arg\max_c \left[\log\dfrac{N_c}{N}+\sum_{i=1}^k \left(x_{ni}\log\mu_{ic}+(1-x_{ni})\log(1-\mu_{ic})\right)\right]\end{aligned}$

In [868]:
class NaiveBayes:
    def __init__(self, pseudo = 1):
        self.pseudo = pseudo
        
    def _binarize(self, X):
        return (X>0).astype(np.int32)
        
    def fit(self, X, y):
        X = self._binarize(X)
        n, k = X.shape
        C = len(np.unique(y)) # number of classes
        self.theta = np.zeros([C,k]) # mean of each feature per class
        self.prior = np.zeros(C)
        for c in range(C):
            self.theta[c] = (X[y==c].sum()+self.pseudo) / (sum(y==c)+self.pseudo*C)
            self.prior[c] = sum(y==c) / n
    
    def predict(self, X):
        X = self._binarize(X)
        log_prior = np.log(self.prior)
        log_post = X@np.log(self.theta.T) + (1-X)@np.log(1-self.theta.T) + log_prior
        return log_post.idxmax(axis=1)
    
    def predict_error(self, X, y):
        return sum(self.predict(X) != y)/y.size

In [869]:
clfNB = NaiveBayes()
clfNB.fit(Xtrain,ytrain)

1. **training error**:

In [870]:
clfNB.predict_error(Xtrain,ytrain)

0.11256117455138662

2. **test error**:

In [871]:
clfNB.predict_error(Xtest,ytest)

0.11002604166666667

### b. GaussianNB

Under **MLE**: $\hat \mu_{ck} = \dfrac{1}{N_c}\sum_{i=1}^Nx_{ik}{\mathbb 1}_{y_i=c},\ {\hat \sigma}_{ck}^2=\dfrac{1}{N_c}\sum_{i=1}^N(x_{ik}-\hat\mu_{ck})^2{\mathbb 1}_{y_i=c}$

In [917]:
from scipy.stats import multivariate_normal

In [814]:
class GaussianNB:
    def __init__(self, transform = None):
        self.transform = transform
        
    def _transform(self, X):
        if self.transform == "stnd":
            return (X-X.mean())/X.std()
        elif self.transform == "log":
            return np.log(X+0.1)   
        return X
    
    def fit(self,X,y):
        X = self._transform(X)
        n, k = X.shape
        self.C = len(np.unique(y)) # number of class
        self.theta = np.zeros([self.C,k]) # mean of each feature per class
        self.sigma = np.zeros([self.C,k]) # variance of each feature per class
        self.prior = np.zeros(self.C)
        for c in range(self.C):
            self.theta[c] = X[y==c].sum() / sum(y==c)
            self.sigma[c] = X[y==c].var(ddof=0)
            self.prior[c] = sum(y==c) / n
        
    def predict(self,X):
        X = self._transform(X)
        log_prior = np.log(self.prior)
        log_post = np.zeros([self.C,X.shape[0]])
        for i in range(self.C):
            log_post[i] = multivariate_normal.logpdf(X, self.theta[i], np.diag(self.sigma[i]))
        log_post =  log_post.T + log_prior
        return np.argmax(log_post,axis=1)
        
    def predict_error(self,X,y):
        return sum(self.predict(X) != y)/y.size

#### b.i) Standardize

In [824]:
clfGNB_stnd = GaussianNB(transform = "stnd")
clfGNB_stnd.fit(Xtrain,ytrain)

1. **training error**:

In [825]:
clfGNB_stnd.predict_error(Xtrain,ytrain)

0.17585644371941273

2. **test error**:

In [826]:
clfGNB_stnd.predict_error(Xtest,ytest)

0.18880208333333334

#### b.ii) Logarithmize

In [822]:
clfGNB_log = GaussianNB(transform = "log")
clfGNB_log.fit(Xtrain,ytrain)

1. **training error**:

In [821]:
clfGNB_log.predict_error(Xtrain,ytrain)

0.1634584013050571

2. **test error**:

In [820]:
clfGNB_log.predict_error(Xtest,ytest)

0.18098958333333334

---
The End